# 0.0. Imports

In [3]:
import pandas                      as pd
import numpy                       as np
import seaborn                     as sns
import inflection
import math
import datetime

from boruta                        import BorutaPy
from sklearn.ensemble              import RandomForestRegressor
from sklearn.linear_model          import LinearRegression, Lasso
from sklearn.preprocessing         import RobustScaler, MinMaxScaler, LabelEncoder
from scipy.stats                   import chi2_contingency
from scipy                         import stats  as ss
from matplotlib                    import pyplot as plt
from IPython.display               import Image
from matplotlib                    import gridspec
from IPython.core.display          import HTML
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## 0.1. Helper functions

In [4]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [35, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [5]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

In [6]:
# load data
df_raw = pd.read_csv('dataset/kc_house_data.csv', low_memory=False) 

# 1.0. Data Preparation

In [7]:
df1 = df_raw.copy()

In [8]:
df1.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 1.1. Data Dimension

In [9]:
# verify number of rows and columns in the dataset

print('Number of rows: {}'.format (df1.shape[0]))
print('Number of Cols: {}'.format (df1.shape[1]))

Number of rows: 21613
Number of Cols: 21


## 1.3. Data types

In [10]:
# change date object to datetime
df1['date'] = pd.to_datetime(df1['date'])

In [11]:
df1.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

## 1.4. Number of NA

In [12]:
# show sum of NA 
df1.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## 1.5. Descriptive Statistics

In [18]:
# Separete numeric and categorical variables
num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64', 'datetime64[ns]'])

print( 'Num of Categorical : {}'.format( cat_attributes.shape[1]) )
print( 'Num of Numerical: {}'.format( num_attributes.shape[1] ) )

Num of Categorical : 0
Num of Numerical: 20


In [19]:
# Central Tendency - Mean, Median

#Dispersion - std deviation, min, max, range, skew, kurtosis



In [15]:
num_atributtes.sample(2)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2190,8644400060,568000.0,3,2.50,2320,57063,1.0,0,0,4,9,1790,530,1979,0,98074,47.6163,-122.056,3290,7314
13788,3840700593,345000.0,3,1.75,1380,10529,1.0,0,0,3,7,1380,0,1967,0,98034,47.7119,-122.233,1670,5694


""
13269
4085
